# Building and Training NMF Model Training Encoder

In [1]:
import boto3,os
import sagemaker as sage, get_execution_role
from time import gmtime,strftime



In [2]:
region = boto3.session.Session().region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')
ecr_repository = 'sagemaker-nmf-container'
tag = ':nmflatest'
training_repository_uri = f'{account_id}.dkr.ecr.{region}.amazonaws.com/{ecr_repository}'

role = get_execution_role()
sess = sage.Session()

In [109]:
!docker build -t $ecr_repository .

Sending build context to Docker daemon  30.72kB
Step 1/10 : FROM python:3.7-slim-buster
 ---> b142ab477505
Step 2/10 : MAINTAINER ASC "mbod@asc.upenn.edu jmparelman@gmail.com"
 ---> Using cache
 ---> 9af566ff1024
Step 3/10 : RUN mkdir -p /data/input /data/output
 ---> Using cache
 ---> 8ec4f649154c
Step 4/10 : RUN apt-get -y update && apt-get install -y --no-install-recommends         build-essential         python-dev         python3-pip     && pip3 install -U pip
 ---> Using cache
 ---> f6a2aea3048b
Step 5/10 : RUN     pip3 install -U pandas          joblib          sklearn
 ---> Using cache
 ---> 496ec0a5ad5f
Step 6/10 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 5d5264c66841
Step 7/10 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> 2ac53032f87e
Step 8/10 : ENV PATH="/opt/ml:${PATH}"
 ---> Using cache
 ---> 13bd2ccf2de2
Step 9/10 : COPY scripts/run_nmf.py /opt/ml/run_nmf.py
 ---> ccfdd76a7a36
Step 10/10 : ENTRYPOINT ["python3","/opt/ml/run_nmf.py"]
 ---> Running in

In [110]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 064258348567.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [111]:
!aws ecr create-repository --repository-name $ecr_repository


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'sagemaker-nmf-container' already exists in the registry with id '064258348567'


In [112]:
!docker tag {ecr_repository} "064258348567.dkr.ecr.us-east-1.amazonaws.com/sagemaker-nmf-container:latest"
!docker push "064258348567.dkr.ecr.us-east-1.amazonaws.com/sagemaker-nmf-container:latest"

The push refers to repository [064258348567.dkr.ecr.us-east-1.amazonaws.com/sagemaker-nmf-container]

ed4854ab: Preparing 
928e1a33: Preparing 
fb75ae7f: Preparing 
7a5861f3: Preparing 
6a810ca1: Preparing 
6bb9ffea: Preparing 
663e622b: Preparing 
845af46d: Preparing 
ed4854ab: Pushed lready exists 6kBAlatest: digest: sha256:e1818bb799fc833214f83a04f6eb2d7d43e9ff99aa00a7ef6ef1d822a7e04ce0 size: 2211


In [114]:
nmf = sage.estimator.Estimator( "064258348567.dkr.ecr.us-east-1.amazonaws.com/sagemaker-nmf-container:latest",
                              role,1, 'ml.m4.16xlarge',
                              output_path = 's3://ascsagemaker/JMP_congressional_nmf/models',
                              sagemaker_session=sess,
                              max_run=7200)

# START JOBS

In [ ]:
congress_list = [102]

In [115]:
for congress in congress_list:
    nmf.fit(f's3://ascsagemaker/JMP_congressional_nmf/dtms/{congress}.pkl',wait=False)